# 🧹 Masterclass de Limpieza de Datos con Pandas - Parte 1

## Contenido
1. [Introducción y Setup](#1)
2. [Creación de Datos de Ejemplo](#2)
3. [Exploración Inicial de Datos](#3)
4. [Manejo de Valores Nulos](#4)
5. [Eliminación de Duplicados](#5)
6. [Corrección de Tipos de Datos](#6)

---

### Introducción
En este notebook aprenderemos técnicas avanzadas de limpieza de datos utilizando pandas. La limpieza de datos es una de las tareas más importantes y que más tiempo consume en el análisis de datos, por lo que dominar estas técnicas es fundamental.

<a id='1'></a>
## 1. Introducción y Setup

Primero importamos las bibliotecas necesarias:

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Configuraciones de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

<a id='2'></a>
## 2. Creación de Datos de Ejemplo

Vamos a crear un dataset que simule datos de ventas de una tienda con varios problemas comunes:

In [ ]:
# Función para generar fechas aleatorias con diferentes formatos
def generar_fecha_aleatoria():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2023, 12, 31)
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    
    formatos = [
        '%Y-%m-%d',        # 2023-12-31
        '%d/%m/%Y',        # 31/12/2023
        '%d-%b-%Y',        # 31-Dec-2023
        '%Y.%m.%d',        # 2023.12.31
        '%d-%m-%y'         # 31-12-23
    ]
    
    formato = random.choice(formatos)
    return random_date.strftime(formato)

# Crear categorías de productos con errores típicos
categorias_correctas = ['Electrónica', 'Ropa', 'Hogar', 'Deportes', 'Libros']
categorias_con_errores = {
    'Electrónica': ['electronica', 'Electronica', 'ELECTRONICA', 'Electrnica', 'Electronca'],
    'Ropa': ['ropa', 'ROPA', 'Roppa', 'Rops', 'Vestimenta'],
    'Hogar': ['hogar', 'HOGAR', 'Hogr', 'Casa', 'HOME'],
    'Deportes': ['deportes', 'DEPORTES', 'Deports', 'Sports', 'Deportivo'],
    'Libros': ['libros', 'LIBROS', 'Libreria', 'Books', 'Libross']
}

# Generar datos
n_registros = 1000
data = {
    'fecha_venta': [generar_fecha_aleatoria() for _ in range(n_registros)],
    'id_producto': [f'PROD_{random.randint(100, 999)}' for _ in range(n_registros)],
    'categoria': [],
    'precio': [round(random.uniform(10, 1000), 2) for _ in range(n_registros)],
    'cantidad': [random.randint(1, 10) for _ in range(n_registros)],
    'id_cliente': [f'CLI_{random.randint(1000, 9999)}' for _ in range(n_registros)],
    'rating': [random.randint(1, 5) if random.random() > 0.2 else None for _ in range(n_registros)]
}

# Generar categorías con errores
for _ in range(n_registros):
    categoria_correcta = random.choice(categorias_correctas)
    if random.random() < 0.7:  # 70% de probabilidad de tener un error
        categoria = random.choice(categorias_con_errores[categoria_correcta])
    else:
        categoria = categoria_correcta
    data['categoria'].append(categoria)

# Crear DataFrame
df = pd.DataFrame(data)

# Introducir algunos valores nulos
df.loc[random.sample(range(n_registros), 50), 'precio'] = None
df.loc[random.sample(range(n_registros), 30), 'id_cliente'] = None

# Introducir algunos duplicados
duplicados = df.sample(n=50)
df = pd.concat([df, duplicados])

# Mostrar las primeras filas del DataFrame
print("Dimensiones del DataFrame:", df.shape)
df.head(10)

<a id='3'></a>
## 3. Exploración Inicial de Datos

Antes de comenzar con la limpieza, es importante entender nuestros datos:

In [ ]:
# Información general del DataFrame
print("\n=== Información General ===")
df.info()

# Estadísticas descriptivas
print("\n=== Estadísticas Descriptivas ===")
print(df.describe())

# Valores nulos por columna
print("\n=== Valores Nulos por Columna ===")
print(df.isnull().sum())

# Valores únicos en categorías
print("\n=== Valores Únicos en Categoría ===")
print(df['categoria'].value_counts())

# Muestra de diferentes formatos de fecha
print("\n=== Muestra de Formatos de Fecha ===")
print(df['fecha_venta'].head(10))

<a id='4'></a>
## 4. Manejo de Valores Nulos

Vamos a tratar los valores nulos de diferentes maneras según la columna:

In [ ]:
# Crear una copia del DataFrame original
df_limpio = df.copy()

# 1. Para precio: rellenar con la mediana
mediana_precio = df_limpio['precio'].median()
df_limpio['precio'] = df_limpio['precio'].fillna(mediana_precio)

# 2. Para rating: rellenar con 0 (asumiendo que la falta de rating significa que no se evaluó)
df_limpio['rating'] = df_limpio['rating'].fillna(0)

# 3. Para id_cliente: crear una categoría 'UNKNOWN'
df_limpio['id_cliente'] = df_limpio['id_cliente'].fillna('CLI_UNKNOWN')

# Verificar que no quedan valores nulos
print("Valores nulos después de la limpieza:")
print(df_limpio.isnull().sum())

<a id='5'></a>
## 5. Eliminación de Duplicados

Vamos a identificar y eliminar registros duplicados:

In [ ]:
# Verificar duplicados exactos
print("Número de duplicados exactos:", df_limpio.duplicated().sum())

# Mostrar algunos ejemplos de duplicados
print("\nEjemplos de registros duplicados:")
print(df_limpio[df_limpio.duplicated(keep=False)].sort_values(by='id_producto').head())

# Eliminar duplicados
df_limpio = df_limpio.drop_duplicates()

print("\nDimensiones después de eliminar duplicados:", df_limpio.shape)

<a id='6'></a>
## 6. Corrección de Tipos de Datos

Vamos a asegurarnos de que cada columna tenga el tipo de dato correcto:

In [ ]:
# Mostrar tipos de datos actuales
print("Tipos de datos antes de la corrección:")
print(df_limpio.dtypes)

# Convertir precio a float
df_limpio['precio'] = pd.to_numeric(df_limpio['precio'], errors='coerce')

# Convertir cantidad a int
df_limpio['cantidad'] = df_limpio['cantidad'].astype(int)

# Convertir rating a int
df_limpio['rating'] = df_limpio['rating'].astype(int)

print("\nTipos de datos después de la corrección:")
print(df_limpio.dtypes)

# Mostrar un resumen del DataFrame limpio
print("\nResumen del DataFrame limpio:")
df_limpio.head()

## Conclusiones de la Parte 1

En esta primera parte hemos cubierto:
1. Creación de un dataset con problemas comunes
2. Exploración inicial de datos
3. Manejo de valores nulos
4. Eliminación de duplicados
5. Corrección de tipos de datos

En la Parte 2 continuaremos con:
- Limpieza de fechas con regex
- Estandarización de categorías
- Detección y manejo de outliers
- Técnicas avanzadas de limpieza